In [3]:
import json
from shapely.geometry import shape, Point

#to plot polygons
import matplotlib.pyplot as plt
import geopandas as gpd

#to open the csv, excel files as dataframes 
import pandas as pd
import numpy as np

#to list all files in a dir
from os import listdir
from os.path import isfile, join

def check_single(geojs, point):
    for feature in geojs['features']:
        polygon = shape(feature['geometry'])
        #print(polygon)
        #p = gpd.GeoSeries(polygon)
        #p.plot()
        #plt.show()
        if polygon.contains(point):
            return True
        
    return False

def check(geofilesdir, spatialfilesdir):
    
    geofiles = [f for f in listdir(geofilesdir) if isfile(join(geofilesdir, f))]
    spatialfiles = [f for f in listdir(spatialfilesdir) if isfile(join(spatialfilesdir, f))]
    print(geofiles)
    print(spatialfiles)
    
    for spatialfile in spatialfiles:
        print("Processing now: " + spatialfile)
        df = pd.read_csv(spatialfilesdir+spatialfile)
        df["school_zones"] = ""
        
        for geofile in geofiles:
            with open(geofilesdir+geofile, 'r') as f:
                geojs = json.load(f)
            schoolname = geofile[:len(geofile) - 8]
            
            for index, row in df.iterrows():
                longitude = float(row['LONGITUDE'])
                latitude = float(row['LATITUDE'])
                point = Point(longitude, latitude)
                zones = row["school_zones"]
                if check_single(geojs, point):
                    zones = zones + schoolname + ","
                df.at[index,'school_zones'] = zones
        
        for index, row in df.iterrows():
            zones = row["school_zones"]
            zones = zones[:-1]
            df.at[index,'school_zones'] = zones
        
        newspatialfile = spatialfile
        df.to_csv(spatialfilesdir+newspatialfile)

In [4]:
#single example: Forest hills clubhouse contained in the CC Spaulding school zone
point = Point(-78.91356462, 35.97993926)
with open('data/2021/map_data/C.C. Spaulding Elementary.geojson', 'r') as f:
    geojs = json.load(f)
    
df = pd.read_csv("data/2021/spatial_data/renamed_Recreation Centers.csv")
point2 = Point(df.iloc[[11]]["LONGITUDE"], df.iloc[[11]]["LATITUDE"])

#should print True
print(check_single(geojs, point))
print(check_single(geojs, point2))

True
True


In [5]:
#updates the .csv files in spatial_data for all the specified years by adding a column 'school_zones' which contains all the
#school zones that resource is in.

years = ['2021']

for year in years:
    geofilesdir = 'data/'+year+'/map_data/'
    spatialfilesdir = 'data/'+year+'/spatial_data/'
    
    check(geofilesdir, spatialfilesdir)

['All.geojson', 'C.C. Spaulding Elementary.geojson', 'E.K. Powe Elementary.geojson', 'Eastway Elementary.geojson', 'Fayetteville Street Elementary.geojson', 'Forest View Elementary.geojson', 'Hillside High.geojson', 'Jordan High.geojson', 'Lakewood Elementary.geojson', 'Parkwood Elementary.geojson', 'Southwest Elementary.geojson']
['renamed_After-School Care Programs.csv', 'renamed_Bus Stops.csv', 'renamed_Childcare Centers_2022.csv', 'renamed_Community & Cultural Centers.csv', 'renamed_Community Gardens.csv', 'renamed_Food Pantries.csv', 'renamed_Grocery Stores.csv', 'renamed_Hospitals and Clinics.csv', 'renamed_Libraries.csv', 'renamed_Parks.csv', 'renamed_Recreation Centers.csv', 'renamed_Religious Centers.csv', 'schools.csv', "url_Farmer's Markets.csv"]
Processing now: renamed_After-School Care Programs.csv
Processing now: renamed_Bus Stops.csv
Processing now: renamed_Childcare Centers_2022.csv
Processing now: renamed_Community & Cultural Centers.csv
Processing now: renamed_Communi